GPT2 generation without lora

too slow, about 8 hours to train

In [1]:
from transformers import GPT2Tokenizer, AdamW, get_scheduler, GPT2LMHeadModel, DataCollatorWithPadding, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
from tqdm import tqdm_notebook as tqdm
import numpy as np
import evaluate
import loralib as lora
import mylora
import re
import time
from matplotlib import pyplot as plt 
%matplotlib inline
myrank=32

In [1]:
model_checkpoint = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "left"

raw_dataset = load_dataset("e2e_nlg_cleaned")
def mytokenize(example):
    ans = []
    for i, rep in enumerate(example["meaning_representation"]):
        ans.append("Write a restaurant description for the following attributes:\n" + rep + "\n" + "Description: " + example["human_reference"][i])
    return tokenizer(ans, truncation=True, padding="max_length", max_length=1024)

dataset = raw_dataset.map(mytokenize, batched=True, remove_columns=['meaning_representation', 'human_reference'])
'''print(dataset)
print(dataset["train"][0])'''

def collate_fn(batch):
    return {
        'input_ids': torch.tensor([item['input_ids'] for item in batch]),
        'attention_mask': torch.tensor([item['attention_mask'] for item in batch]),
        'labels': torch.tensor([item['input_ids'] for item in batch])  # Labels are input_ids for language modeling
    }

train_loader = DataLoader(dataset['train'], batch_size=4, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset['validation'], batch_size=4, collate_fn=collate_fn)

NameError: name 'GPT2LMHeadModel' is not defined

In [3]:
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 1
num_training_steps = num_epochs * len(train_loader)
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = model.to(device)

rouge_score = evaluate.load("rouge")

def eval_model():
    model.eval()
    preds = []
    labs = []
    num_batches = len(valid_loader)
    progress_bar = tqdm(valid_loader, desc=f"Batch {0}/{num_batches}", unit="batch")

    for i, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            generated_tokens = model.generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            ).cpu().numpy()

        label_tokens = batch["labels"].cpu().numpy()
        label_tokens = np.where(label_tokens != -100, label_tokens, tokenizer.pad_token_id)

        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(label_tokens, skip_special_tokens=True)
        preds += decoded_preds
        labs += decoded_labels
        progress_bar.set_description(f"Batch {i+1}/{num_batches}")

    scores = rouge_score.compute(
        predictions=preds, references=labs
    )
    print(scores)

e:\anaconda3\envs\myml\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


cuda


In [4]:
myloss = 0.0
mylosses = []
num_batches = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}, Batch 0/{num_batches}", unit="batch")
    for i, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model(**batch)
        loss = output.loss
        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        myloss = loss.item() / len(batch)
        mylosses.append(myloss)
        progress_bar.set_description(f"Epoch {epoch+1}, Batch {i+1}/{num_batches}")
    print("epoch "+str(epoch+1)+", evaluate model")
    eval_model()

C:\Users\Developer\AppData\Local\Temp\ipykernel_25128\3143187867.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}, Batch 0/{num_batches}", unit="batch")


Epoch 1, Batch 0/8382:   0%|          | 0/8382 [00:00<?, ?batch/s]

KeyboardInterrupt: 